#### Aim: Match patient from ClinCapture data with software data
#### Intput: Software database, clinical database (in Mysql)
#### Output: 
- Store one patient's complete clinical information into his(her) related image data folder
- Summary table for all patients form both databases is included


In [3]:
import numpy as np
import pandas as pd

In [4]:
# connect softwaredata to mysql
import mysql.connector
from mysql.connector import errorcode
try:
  config={
    'user': 'root',
    'password': '123456',
    'host': 'localhost',
    'database': 'softwaredata',
    'raise_on_warnings': True,
    }

  cnx1= mysql.connector.connect(**config)

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

        

In [237]:
# connect clinical information to mysql
import mysql.connector
from mysql.connector import errorcode
try:
  config={
    'user': 'root',
    'password': '123456',
    'host': 'localhost',
    'database': 'clinical_info',
    'raise_on_warnings': True,
    }


  cnx= mysql.connector.connect(**config)

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

### Clinical info


In [288]:
# get the information from the clinical information
cur=cnx.cursor()

cur.execute("SELECT subject_enrollment_number1,initials,gender,dob FROM clinical_info.enrollment_raw;" )
result = cur.fetchall()

full_ini = pd.DataFrame(result)


In [289]:
col_name=['subjectnum','Ini_name',"Gender","DOB"]
full_ini.columns=col_name
full_ini.index = range(1,len(full_ini)+1)


In [291]:
# Just get the first letter of firstname and lastname, in order to match the names from software data
for i in range(1,len(full_ini)+1):
    new_ini = full_ini.ix[i,"Ini_name"][0]+full_ini.ix[i,"Ini_name"][-1]
    full_ini.set_value(i, "Initial", new_ini)
full_ini.head()

,subjectnum,Ini_name,Gender,DOB,Initial
1,0001-001,DR,0.0,19580402,DR
2,0001-002,JG,0.0,19470210,JG
3,0001-003,DJ,0.0,19460212,DJ
4,0001-004,PD,1.0,19410512,PD
5,0001-005,EIS,0.0,19470223,ES


#### Imaging date from clinic database

In [241]:
# get the information from the clinical information
cur_img=cnx.cursor()

cur_img.execute("SELECT subjectnum,imagingdate FROM clinical_info.imaging_raw order by subjectnum;" )
result_img = cur_img.fetchall()

result_img = pd.DataFrame(result_img)
for i in range(len(result_img)):
    if str(result_img.ix[i][0])[6]=='0':
        result_img.ix[i][0]= int(result_img.ix[i][0][7])
    else:
        result_img.ix[i][0]= int(result_img.ix[i][0][6:8])

In [292]:
result_img.head()

,0,1
0,1,20160408
1,1,20160415
2,2,20160408
3,3,20160408
4,3,20160422


In [70]:
imgdate = pd.DataFrame(index=range(1,len(full_ini)+1), columns=["img1","img2"])
imgdate = imgdate.fillna(0)

In [293]:
for i in range(0,len(result_img)-2):
    if result_img.ix[i][0] == result_img.ix[i+1][0]:
        imgdate.ix[result_img.ix[i][0]][0]=int(result_img.ix[i][1])
        imgdate.ix[result_img.ix[i][0]][1]=int(result_img.ix[i+1][1])

    else:
        imgdate.ix[result_img.ix[i+1][0]][0]=int(result_img.ix[i+1][1])
        
        
imgdate.head()

,img1,img2
1,20160408,20160415
2,20160408,0
3,20160408,20160422
4,20160408,20160520
5,20160411,0


In [294]:
df_clinic = pd.concat([full_ini,imgdate],axis=1, join_axes=[full_ini.index])
df_clinic


,subjectnum,Ini_name,Gender,DOB,Initial,img1,img2
1,0001-001,DR,0.0,19580402,DR,20160408,20160415
2,0001-002,JG,0.0,19470210,JG,20160408,0
3,0001-003,DJ,0.0,19460212,DJ,20160408,20160422
4,0001-004,PD,1.0,19410512,PD,20160408,20160520
5,0001-005,EIS,0.0,19470223,ES,20160411,0
6,0001-006,ARB,0.0,19620810,AB,20160412,0
7,0001-007,CGK,0.0,19440716,CK,20160412,0
8,0001-008,MKG,0.0,19471218,MG,20160412,20160617
9,0001-009,ARP,1.0,19490311,AP,20160415,20160506
10,0001-010,DH,0.0,19510403,DH,20160422,0


### Softwaredata

In [297]:
# SQl query
# get the the firstname and lastname from softwaredata
cur1=cnx1.cursor()

cur1.execute("SELECT FirstName, LastName, Sex, Birthday FROM softwaredata.patient;" )
result1 =  cur1.fetchall()
res= np.asarray(result1)

colnames = []
for k in cur1.column_names:
    colnames.append(k)
colnames[2]="Gender"
colnames[3]="DOB"

In [301]:
swdf=pd.DataFrame(res.T,colnames)
swdft=swdf.T


In [302]:
# delete the fake ones
newdf=swdft
for i in range(len(swdft)):
    if len(swdft.ix[i,0])>1 or len(swdft.ix[i,1]) >1:
        newdf = newdf.drop(swdft.index[[i]])
newdf.head()

,FirstName,LastName,Gender,DOB
3,d,r,male,112736664000000000
4,j,g,male,109221048000000000
5,d,j,male,108907416000000000
6,p,d,female,107406612000000000
7,E,S,male,109232280000000000


In [303]:
# combine the first and second column
newdf["Initial"] = newdf["FirstName"].map(str) + newdf["LastName"]
newdf['Initial'] = map(lambda x: x.upper(), newdf['Initial'])
newdf.ix[newdf.Gender == "male","Gender"]= 0
newdf.ix[newdf.Gender == "female","Gender"]= 1
newdf["DOB"] = map(lambda x:int(x), newdf['DOB'])
newdf.index=range(1,len(newdf)+1)
newdf.head()

,FirstName,LastName,Gender,DOB,Initial
1,d,r,0,112736664000000000,DR
2,j,g,0,109221048000000000,JG
3,d,j,0,108907416000000000,DJ
4,p,d,1,107406612000000000,PD
5,E,S,0,109232280000000000,ES


#### Imaging date  from software data

In [304]:
df_img_soft = pd.DataFrame()
for i in range(len(newdf)):
    cur_img_soft=cnx1.cursor()
    cur_img_soft.execute("SELECT S.SERID,  I.CreateDateTime, S.SID, I.ImageCollFolderName\
    FROM softwaredata.imagescollection I \
    JOIN softwaredata.series S \
    ON S.SERID = I.SERID \
    WHERE S.SERID IN (SELECT SERID FROM softwaredata.series \
    WHERE SID IN (SELECT SID FROM softwaredata.study WHERE PID =%d))\
    GROUP BY SID; "%(i+4))
    ans= cur_img_soft.fetchall()
    df_ans = pd.DataFrame(ans)
    colnames = []
    
    df_ans = pd.DataFrame(ans)  
    df_ans.index = (i+1)*np.ones((len(df_ans),), dtype=np.int)
    df_img_soft = pd.concat([df_img_soft,df_ans],axis=0)


colnames=["SERID","Imgdate","SID","Folder_path"]
df_img_soft.columns=colnames
df_img_soft.head()

,SERID,Imgdate,SID,Folder_path
1,16,131045980405424248,12,D:\SpectralView\Patient4\Diabetic_Ulcers\Study...
1,59,131082416733818147,42,D:\SpectralView\Patient4\Diabetic_Ulcers\Study...
2,17,131046025712015637,13,D:\SpectralView\Patient5\Diabetic_Ulcers\Study...
3,18,131046060622812420,14,D:\SpectralView\Patient6\Chronic_Wounds\Study1...
3,32,131058140499845454,25,D:\SpectralView\Patient6\Diabetic_Ulcers\Study...


In [164]:
imgdate_soft = pd.DataFrame(index=range(1,len(newdf)+1), columns=["img1","img2","img3"])
imgdate_soft = imgdate_soft.fillna(0)

In [305]:
for i in range(0,len(newdf)+1):
    
    if list(df_img_soft.index).count(i)==1:
            imgdate_soft.ix[i][0]=np.asarray(df_img_soft["Imgdate"].ix[i])
    elif list(df_img_soft.index).count(i)==2:
            imgdate_soft.ix[i][0]=np.asarray(df_img_soft["Imgdate"].ix[i])[0]
            imgdate_soft.ix[i][1]=np.asarray(df_img_soft["Imgdate"].ix[i])[1]
    elif list(df_img_soft.index).count(i)==3:
            imgdate_soft.ix[i][0]=np.asarray(df_img_soft["Imgdate"].ix[i])[0]
            imgdate_soft.ix[i][1]=np.asarray(df_img_soft["Imgdate"].ix[i])[1]
            imgdate_soft.ix[i][2]=np.asarray(df_img_soft["Imgdate"].ix[i])[2]
imgdate_soft.head()
        

,img1,img2,img3
1,131045980405424248,131082416733818147,0
2,131046025712015637,0,0
3,131046060622812420,131058140499845454,131094412334083500
4,131046084741789809,131082380503115869,0
5,131048595160179361,0,0


In [306]:
df_soft = pd.concat([newdf,imgdate_soft],axis=1,join_axes=[newdf.index])
df_soft.head()

,FirstName,LastName,Gender,DOB,Initial,img1,img2,img3
1,d,r,0,112736664000000000,DR,131045980405424248,131082416733818147,0
2,j,g,0,109221048000000000,JG,131046025712015637,0,0
3,d,j,0,108907416000000000,DJ,131046060622812420,131058140499845454,131094412334083500
4,p,d,1,107406612000000000,PD,131046084741789809,131082380503115869,0
5,E,S,0,109232280000000000,ES,131048595160179361,0,0


In [307]:
EPOCH_AS_FILETIME=116444736000000000
def filetime_to_dt(ft):
    us = (ft - EPOCH_AS_FILETIME) // 10
    dt= datetime(1970, 1, 1) + timedelta(microseconds = us)
    if len(str(dt.month)) ==1:
        month = "0"+str(dt.month)
    else:
        month = str(dt.month)
    if len(str(dt.day)) ==1:
        date1 = "0"+str(dt.day)
    else:
        date1 = str(dt.day)
        
    return str(dt.year)+""+month+""+date1


In [308]:
df_soft["img1"] = map(lambda x:filetime_to_dt(long(x)), df_soft['img1'])

df_soft["img2"] = map(lambda x:filetime_to_dt(long(x)), df_soft['img2'])

df_soft["img3"] = map(lambda x:filetime_to_dt(long(x)), df_soft['img3'])

df_soft["DOB"] = map(lambda x:filetime_to_dt(long(x)), df_soft['DOB'])

In [309]:
df_soft

,FirstName,LastName,Gender,DOB,Initial,img1,img2,img3
1,d,r,0,19580402,DR,20160408,20160520,16010101
2,j,g,0,19470210,JG,20160408,16010101,16010101
3,d,j,0,19460212,DJ,20160408,20160422,20160603
4,p,d,1,19410512,PD,20160408,20160520,16010101
5,E,S,0,19470223,ES,20160411,16010101,16010101
6,a,b,0,19620810,AB,20160412,16010101,16010101
7,c,k,0,19440716,CK,20160412,16010101,16010101
8,m,g,0,19471218,MG,20160412,16010101,16010101
9,r,d,0,19580402,RD,20160415,16010101,16010101
10,a,p,1,19490311,AP,20160415,20160506,20160520


### Match clinic data with software data

- Add weight on different features
- Add threshold 

In [210]:
index = []
for i in range(len(df_clinic)):
    
    sex = df_soft["Gender"]==df_clinic["Gender"].ix[i+1]
    ini = df_soft["Initial"] == df_clinic["Initial"].ix[i+1]
    dob = df_soft["DOB"] == df_clinic["DOB"].ix[i+1]
    img1 = df_soft["img1"] == df_clinic["img1"].ix[i+1]
    img2= df_soft["img2"] == df_clinic["img2"].ix[i+1]
    
    score_mat = np.concatenate((np.asarray([ini]).T,\
               np.asarray([dob]).T,np.asarray([sex]).T,np.asarray([img1]).T,\
                np.asarray([img2]).T),axis=1)

    df_score = pd.DataFrame(score_mat)
    df_score.index = range(1,len(df_score)+1)
    df_score.columns=["Initial","DOB","Gender","Img_visit1","Img_visit2"]
    df_score['score'] = df_score['Initial']*0.25+df_score['DOB']*0.35+\
df_score['Gender']*0.1+df_score['Img_visit1']*0.15+df_score['Img_visit2']*0.15
    if df_score['score'][np.argmax(df_score['score'])]>0.4:
        index.append(np.argmax(df_score['score']))
    else:
        index.append(0)
    

In [211]:
index

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 0, 19, 20, 0]

In [212]:
df_score

,Initial,DOB,Gender,Img_visit1,Img_visit2,score
1,False,False,True,False,False,0.1
2,False,False,True,False,False,0.1
3,False,False,True,False,False,0.1
4,False,False,False,False,False,0.0
5,False,False,True,False,False,0.1
6,False,False,True,False,False,0.1
7,False,False,True,False,False,0.1
8,False,False,True,False,False,0.1
9,False,False,True,False,False,0.1
10,False,False,False,False,False,0.0


In [263]:
# connect clinical information to mysql
import mysql.connector
from mysql.connector import errorcode
try:
  config={
    'user': 'root',
    'password': '123456',
    'host': 'localhost',
    'database': 'clinical_info',
    'raise_on_warnings': True,
    }


  cnx= mysql.connector.connect(**config)

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [287]:
# Find the corresponding clinical information and store into the same folder with their image data


import os 

for l in range(len(index)):
    
    if l+1<10:
        num = "0001-00"+str(l+1)
    else:
        num = "0001-0"+str(l+1)
        
    cur=cnx.cursor()

    cur.execute("SELECT * FROM clinical_info.enrollment_raw LEFT JOIN clinical_info.imaging_raw \
ON clinical_info.enrollment_raw.subject_enrollment_number1=clinical_info.imaging_raw.subjectnum \
WHERE clinical_info.enrollment_raw.subject_enrollment_number1='%s'" %(num))

    result = cur.fetchall()
   
    col=cur.column_names
    
    res= np.asarray(result)
    
    colnames = []
    for k in col:
        colnames.append(k)
    pat=pd.DataFrame(res.T,colnames)    

    if index[l]!= 0:
        path =  str(df_img_soft["Folder_path"].ix[index[l]]).split("\\")
        if os.path.exists('E:/imagedata/'+path[2]+'/patient_info_update.csv'):
            pat.to_csv('E:/imagedata/'+path[2]+'/patient_info_update.csv' ,header = False)
        else:
            print("Patient "+num+" matched, but because the software data folder not updated simutaneously,we could not find the corrected folder to save the results")
        
   
    else: 
    
        print("Patient "+num+" from clinical data mismathched with patients in software data")
    

Patient 0001-017 from clinical data mismathched with patients in software data
Patient 0001-018 matched, but because the software data folder not updated simutaneously,            we could not find the corrected folder to save the results
Patient 0001-019 matched, but because the software data folder not updated simutaneously,            we could not find the corrected folder to save the results
Patient 0001-020 from clinical data mismathched with patients in software data


### Summary table


In [332]:
patient_SMD= pd.DataFrame(index=range(1,len(df_clinic)+1),columns=['patient_smd'])
for i in range(len(patient_SMD)):
    if index[i]!=0:
        path =  str(df_img_soft["Folder_path"].ix[index[i]]).split("\\")
        patient_SMD['patient_smd'].ix[i+1]=path[2]


In [333]:
summary_tb = pd.concat([df_clinic[['subjectnum','Initial','DOB']], \
                        patient_SMD],axis=1, join_axes=[df_clinic.index])
summary_tb

,subjectnum,Initial,DOB,patient_smd
1,0001-001,DR,19580402,Patient4
2,0001-002,JG,19470210,Patient5
3,0001-003,DJ,19460212,Patient6
4,0001-004,PD,19410512,Patient7
5,0001-005,ES,19470223,Patient8
6,0001-006,AB,19620810,Patient9
7,0001-007,CK,19440716,Patient10
8,0001-008,MG,19471218,Patient11
9,0001-009,AP,19490311,Patient13
10,0001-010,DH,19510403,Patient14


In [334]:
summary_tb.to_csv('E:/imagedata/summary_tb.csv',header= True)